#### Towards implementing Natural Join
CS 236 <br>
Fall 2023

Michael A. Goodrich <br>
Brigham Young University <br>
March 2023
***

Project 4 has historically been the most challenging project for students. The purpose of this Jupyter notebook is to make a better connection between the high-level discussion of natural join from class and an implementation of natural join.

We'll approach this by implementing the pseudo-code in the Project 4 description in two special cases.
***

#### Pseudo-code from project 4 description and class slides ####
Special case when the headers of the two relations are identical.
<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
1 create header <font color = "lightblue">my_header</font><br>
2 create empty relation <font color = "lightblue">new_relation</font> with header <font color = "lightblue">my_header</font><br>
3 for each tuple <font color = "lightblue">tuple_1</font> in <font color = "lightblue">relation_1</font><br>
4 &nbsp;&nbsp;&nbsp;&nbsp; for each tuple <font color = "lightblue">tuple_2</font> in <font color = "lightblue">relation_2</font><br>
5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if <font color = "lightblue">tuple_1</font> and <font color = "lightblue">tuple_2</font> can join <br>
6 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<font color = "lightblue">new_tuple</font> = <font color = "red">join</font>(<font color = "lightblue">tuple_1</font>,<font color = "lightblue">tuple_2</font>)<br>
7 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
add <font color = "lightblue">new_tuple</font> to <font color = "lightblue">new_relation</font>
</div>


***

#### Special Case 1: Identical Attributes ####

Natural join is equivalent to intersection if the attributes of the two relations are identical. Let's see what this looks like in the pseudo-code. We'll begin by copying the relation class from the previous Jupyter notebook. 

Consider the relation R1 defined as 
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ | 

and consider a new relation R2 defined as
| char | int | 
| :-: | :-: | 
| $f$ | $3$ |
| $b$ | $2$ | 
| $b$ | $3$ | 
| $d$ | $4$ | 

According to class lecture, when the headers of the two relations match the output $R1 \bowtie R2$ is equivalent to $R1 \cap R2$, which yields
| char | int | 
| :-: | :-: | 
| $b$ | $2$ | 
| $d$ | $4$ | 


In [14]:
from tabulate import tabulate
class Relation:
    def __init__(self, relation_name, relation_header, set_of_tuples):
        self.name = relation_name # I threw in a string for the relation name for fun
        self.header = relation_header
        self.set_of_tuples = set_of_tuples

    ########################
    # Relational Operators #
    ########################
    def NaturalJoin__SameHeaders(self,other):
        if type(other) != Relation: raise TypeError
        # Check whether conditions for special case apply
        if self.getHeader() != other.getHeader(): raise ValueError
        # Pseudo-code line 1: create the header
        header = self.getHeader()
        # Pseudo-code line 2: create an empty relation
        new_name = self.getName() + " \u2A1D " + other.getName() # The butterfly represents natural join
        new_tuple_set = set()
        new_relation = Relation(new_name,header,new_tuple_set)
        for tuple_1 in self.getTuples():        # Pseudo-code line 3
            for tuple_2 in other.getTuples():   # Pseudo-code line 4
                if tuple_1 == tuple_2:          # Pseudo-code line 5. The condition for join is equality
                    new_tuple = tuple_1         # Pseudo-code line 6
                    new_relation.addTuple(new_tuple) # Pseudo-code line 7
        return new_relation
    
    ####################
    # Public Utilities #
    ####################
    def addTuple(self,tuple):
        if len(self.header) != len(tuple): raise ValueError
        self.set_of_tuples.add(tuple)
    def toString(self):
        ### Prints the name, header, and contents of the relation
        print("The relation name is ", self.name)
        print(tabulate(self.set_of_tuples,headers = self.header,tablefmt = 'fancy_grid'))

    
    #######################
    # Getters and Setters #
    #######################
    def getName(self): return self.name
    def getHeader(self): return self.header
    def getTuples(self): return self.set_of_tuples

    ####################
    # System Utilities #
    ####################
    # Define how the == operator acts on relations #
    def __eq__(self,other):
        # First, check whether the thing passed to the equality method is the same type
        if not isinstance(other, Relation):
            # don't attempt to compare against unrelated types
            raise ValueError
        # Second, return true only if the header and sets all match. I don't really care if the names match
        return self.header == other.header and self.set_of_tuples == other.set_of_tuples

R1 = Relation(relation_name = 'R',relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
R2 = Relation('Q',('char','int'),{('f',3),('b',2),('b',3),('d',4)}) 
P = R1.NaturalJoin__SameHeaders(R2)
P.toString()

The relation name is  R ⨝ Q
╒════════╤═══════╕
│ char   │   int │
╞════════╪═══════╡
│ b      │     2 │
├────────┼───────┤
│ d      │     4 │
╘════════╧═══════╛


Yay! It worked.

Let's consider a second special case, one in which exactly one attribute matches. The pseudo-code looks like this:

#### Pseudo-code from project 4 description and class slides ####
General case, but only implemented for the case when exactly one attribute differs between the two relations.
<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
1 create header <font color = "lightblue">my_header</font> by <font color = "red">joining</font> <font color = "lightblue">header_1</font> and <font color = "lightblue">header_2</font><br>
2 create empty relation <font color = "lightblue">new_relation</font> with header <font color = "lightblue">my_header</font><br>
3 for each tuple <font color = "lightblue">tuple_1</font> in <font color = "lightblue">relation_1</font><br>
4 &nbsp;&nbsp;&nbsp;&nbsp; for each tuple <font color = "lightblue">tuple_2</font> in <font color = "lightblue">relation_2</font><br>
5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if <font color = "lightblue">tuple_1</font> and <font color = "lightblue">tuple_2</font> can join <br>
6 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<font color = "lightblue">new_tuple</font> = <font color = "red">join</font>(<font color = "lightblue">tuple_1</font>,<font color = "lightblue">tuple_2</font>)<br>
7 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
add <font color = "lightblue">new_tuple</font> to <font color = "lightblue">new_relation</font>
</div>

***
We'll create two private methods for the relation class.
* __joinHeaders(self,other)
* __jointTuples(self,tuple_1,tuple_2)

These two private methods will be designed to work only for the special case. You'll need to extend these for Project 4.

Here are the relations to which we'll apply.

Relation R1 is defined as
| A | B | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 

and relation R2 is defiend as
| A | D | 
| :-: | :-: | 
| $a$ | $1$ |
| $a$ | $4$ |
| $c$ | $5$ |
| $d$ | $3$ | 

From the class slides, the natural join $R1 \bowtie R2$ is
| A | B | D |
| :-: | :-: | :-: |
| $a$ | $1$ | $1$ |
| $a$ | $1$ | $4$ |
| $c$ | $3$ | $5$ |

***
In the code below, look at three methods: the new natural join method, the private __joinHeaders, the private method __canTuplesJoin, and the private method __joinTuples. There are a few other small changes, like pushing the check (to see whether the precondition for this special case of natural join) into the helper function __findCommonAttribute 

In [44]:
class Relation:
    def __init__(self, relation_name, relation_header, set_of_tuples):
        self.name = relation_name # I threw in a string for the relation name for fun
        self.header = relation_header
        self.set_of_tuples = set_of_tuples

    ########################
    # Relational Operators #
    ########################
    def NaturalJoin(self,other):
        # Essentially the same code except for how the header is created and the tuples are joined
        if type(other) != Relation: raise TypeError
        # Pseudo-code line 1: create the header
        new_header,common_attribute_index = self.__joinHeaders(other) # Moved the check of the special condition to this method
        # Pseudo-code line 2: create an empty relation
        new_name = self.getName() + " \u2A1D " + other.getName() # The butterfly represents natural join
        new_tuple_set = set()
        new_relation = Relation(new_name,new_header,new_tuple_set)
        for tuple_1 in self.getTuples():        # Pseudo-code line 3
            for tuple_2 in other.getTuples():   # Pseudo-code line 4
                if self.__canTuplesJoin(tuple_1,tuple_2,common_attribute_index):          # Pseudo-code line 5. The condition for join is equality
                    new_tuple = self.__joinTuples(tuple_1,tuple_2,common_attribute_index)  # Pseudo-code line 6
                    new_relation.addTuple(new_tuple) # Pseudo-code line 7
        return new_relation
    
    ####################
    # Public Utilities #
    ####################
    def addTuple(self,tuple):
        self.set_of_tuples.add(tuple)
    def toString(self):
        ### Prints the name, header, and contents of the relation
        print("The relation name is ", self.name)
        print(tabulate(self.set_of_tuples,headers = self.header,tablefmt = 'fancy_grid'))

    ############################
    # Private Helper Functions #
    ############################
    def __findCommonAttribute(self,header_1,header_2):
        # The following code is written for clarity and not for speed
        number_common_attributes = 0
        for attribute_1_index in range(len(header_1)):
            for attribute_2_index in range(len(header_2)):
                if header_1[attribute_1_index] == header_2[attribute_2_index]:
                    common_attribute_index = attribute_1_index
                    number_common_attributes += 1
        if number_common_attributes != 1: raise ValueError # Special case has exactly one matching attribute
        return common_attribute_index
    def __joinHeaders(self,other):
        # The following code is written for clarity and not for speed
        header_1 = self.getHeader()
        header_2 = other.getHeader()
        common_attribute_index = self.__findCommonAttribute(header_1,header_2)
        new_header = []
        for attribute_1_index in range(len(header_1)):
            new_header.append(header_1[attribute_1_index])
        for attribute_2_index in range(len(header_2)):
            if attribute_2_index != common_attribute_index:
                new_header.append(header_2[attribute_2_index])
        return new_header,common_attribute_index   
    def __canTuplesJoin(self,tuple_1,tuple_2,common_attribute_index):
        return tuple_1[common_attribute_index] == tuple_2[common_attribute_index]
    def __joinTuples(self,tuple_1, tuple_2, common_attribute_index):
        new_tuple = []
        for index_1 in range(len(tuple_1)):
            new_tuple .append(tuple_1[index_1])
        for index_2 in range(len(tuple_2)):
            if index_2 != common_attribute_index:
                new_tuple .append(tuple_2[index_2])
        return tuple(new_tuple) # Cast list to immutable tuple type

    #######################
    # Getters and Setters #
    #######################
    def getName(self): return self.name
    def getHeader(self): return self.header
    def getTuples(self): return self.set_of_tuples

    ####################
    # System Utilities #
    ####################
    # Define how the == operator acts on relations #
    def __eq__(self,other):
        # First, check whether the thing passed to the equality method is the same type
        if not isinstance(other, Relation):
            # don't attempt to compare against unrelated types
            raise ValueError
        # Second, return true only if the header and sets all match. I don't really care if the names match
        return self.header == other.header and self.set_of_tuples == other.set_of_tuples


In [46]:
R1 = Relation(relation_name = 'R',relation_header = ('A','B'), set_of_tuples = {('a',1),('b',2),('c',3)})
R2 = Relation('Q',('A','D'),{('a',1),('a',4),('c',5),('d',3)}) 
#R1.toString()
#R2.toString()
P = R1.NaturalJoin(R2)
P.toString()

The relation name is  R
╒═════╤═════╕
│ A   │   B │
╞═════╪═════╡
│ b   │   2 │
├─────┼─────┤
│ c   │   3 │
├─────┼─────┤
│ a   │   1 │
╘═════╧═════╛
The relation name is  Q
╒═════╤═════╕
│ A   │   D │
╞═════╪═════╡
│ d   │   3 │
├─────┼─────┤
│ a   │   4 │
├─────┼─────┤
│ c   │   5 │
├─────┼─────┤
│ a   │   1 │
╘═════╧═════╛
The relation name is  R ⨝ Q
╒═════╤═════╤═════╕
│ A   │   B │   D │
╞═════╪═════╪═════╡
│ c   │   3 │   5 │
├─────┼─────┼─────┤
│ a   │   1 │   4 │
├─────┼─────┼─────┤
│ a   │   1 │   1 │
╘═════╧═════╧═════╛
